#  Машинный перевод с использованием рекуррентных нейронных сетей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-seq2seq-model
* https://www.adeveloperdiary.com/data-science/deep-learning/nlp/machine-translation-recurrent-neural-network-pytorch/

## Задачи для совместного разбора

1\. Рассмотрите пример архитектуры Encoder-Decoder с использованием RNN. Обсудите концепцию teacher forcing.

In [ ]:
import torch as th
import torch.nn as nn

In [ ]:
n_ru = 1000
n_en = 500
batch_size=16
seq_len_ru = 15
seq_len_en = 10


ru = th.randint(0, n_ru, size=(batch_size, seq_len_ru))
en = th.randint(0, n_en, size=(batch_size, seq_len_en))

In [ ]:
class Encoder(nn.Module):
  def __init__(self, emb_dim, hidden_size, n_ru):
    super().__init__()
    self.emb = nn.Embedding(num_embeddings=n_ru, embedding_dim=emb_dim, padding_idx=0)
    self.drop = nn.Dropout(p=0.2)
    self.rnn = nn.GRU(emb_dim, hidden_size, batch_first=True)

  def forward(self, X):
    e = self.emb(X)
    out = self.drop(e)
    out, h = self.rnn(out)
    return h # 1 x batch x hidden_size

In [ ]:
emb_dim = 100
hid_size = 300

en = Encoder(emb_dim, hid_size, n_ru)
out = en(ru)
out.shape

torch.Size([1, 16, 300])

In [ ]:
class Decoder(nn.Module):
  def __init__(self, emb_dim, hidden_size, n_en):
    super().__init__()
    self.emb = nn.Embedding(num_embeddings=n_en, embedding_dim=emb_dim, padding_idx=0)
    self.rnn = nn.GRUCell(emb_dim, hidden_size)
    self.fc = nn.Linear(in_features=hidden_size, out_features=n_en)

  def forward(self, encoder_out, labels):
    # labels: batch x seq_len - считаем, что в 0 столбце стоят sos
    # encoder: 1 x batch x hidden_size
    seq_len = labels.size(1)
    input_tokens = labels[:, 0]
    decoder_hidden = encoder_out[0]
    for _ in range(1, len(seq_len)):
      out = self.emb(input_tokens).relu() # batch x emb_dim
      decoder_hidden = self.rnn(out, decoder_hidden) # batch x dec_hidden
      out = self.fc(decoder_hidden) # batch x n_en

      # teacher forcing(надо реализовать(переопределить input_tokens из labels))
      input_tokens = out.argmax(dim=1).detach()

    # вернуть прогнозы для каждого эл-та последовательности
    # batch x seq_len x n_en
    return ...

## Задачи для самостоятельного решения

In [ ]:
!pip install torchmetrics

In [ ]:
import torch as th
import torch.nn as nn
import torch.optim as optim
import torchtext.transforms as T
import torchmetrics as M
from torch.utils.data.dataloader import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import json
import re

<p class="task" id="1"></p>

1\. Считайте файлы `RuBQ_2.0_train.json` (обучающее множество) и `RuBQ_2.0_test.json` (тестовое множество). Для каждого файла создайте по списка: список предложений на русском языке и список предложений на английском языке. Выведите на экран количество примеров в обучающей и тестовой выборке.

- [ ] Проверено на семинаре

In [ ]:
X_train = []
y_train = []
X_test = []
y_test = []

with open('RuBQ_2.0_train.json', 'r') as f:
  for dct in json.load(f):
    X_train.append(re.sub(r'[^a-zа-я ]', '', dct['question_text'].lower()).split())
    y_train.append(re.sub(r'[^a-zа-я ]', '', dct['question_eng'].lower()).split())
with open('RuBQ_2.0_test.json', 'r') as f:
  for dct in json.load(f):
    X_test.append(re.sub(r'[^a-zа-я ]', '', dct['question_text'].lower()).split())
    y_test.append(re.sub(r'[^a-zа-я ]', '', dct['question_eng'].lower()).split())

len(X_train), len(y_train), len(X_test), len(y_test)

(2330, 2330, 580, 580)

In [ ]:
X_train[0], y_train[0]

(['что', 'может', 'вызвать', 'цунами'],
 ['what', 'can', 'cause', 'a', 'tsunami'])

<p class="task" id="2"></p>

2\. Создайте два Vocab на основе загруженных данных: `ru_vocab` для слов на русском языке и `en_vocab` для слов на английском языке (словари создаются на основе обучающего множества). Добавьте в словари специальные токены `<PAD>`, `<SOS>`, `<EOS>`. Выведите на экран количество токенов в полученных словарях. Выведите на экран максимальное кол-во слов в предложениях на русском языке и в предложениях на английском языке (в обучающей выборке).

- [ ] Проверено на семинаре

In [ ]:
ru_vocab = build_vocab_from_iterator(X_train, specials=['<PAD>', '<UNK>', '<SOS>', '<EOS>'])
en_vocab = build_vocab_from_iterator(y_train, specials=['<PAD>', '<UNK>', '<SOS>', '<EOS>'])
ru_vocab.set_default_index(ru_vocab['<UNK>'])
en_vocab.set_default_index(en_vocab['<UNK>'])
len(ru_vocab), len(en_vocab)

(5825, 4234)

In [ ]:
print(f'max length in russian: {max(map(lambda x: len(x), X_train))}')
print(f'max length in english: {max(map(lambda x: len(x), y_train))}')

max length in russian: 25
max length in english: 31


<p class="task" id="3"></p>

3\. Создайте класс `RuEnDataset`. Реализуйте `__getitem__` таким образом, чтобы он возвращал кортеж `(x, y)`, где x - это набор индексов токенов для предложений на русском языке, а `y` - набор индексов токенов для предложений на английском языке. Используя преобразования, сделайте длины наборов индексов одинаковой фиксированной длины, добавьте в начало каждого набора индекс `<SOS>`, а в конец - индекс токена `<EOS>`. Создайте датасет для обучающей и тестовой выборки.

- [ ] Проверено на семинаре

In [ ]:
class RuEnDataset:
  def __init__(self, ru_sents, en_sents, transforms):
    self.ru_sents = ru_sents
    self.en_sents = en_sents
    self.transform_ru = transforms[0]
    self.transform_en = transforms[1]

  def __getitem__(self, idx):
    return self.transform_ru(self.ru_sents[idx]), self.transform_en(self.en_sents[idx])

  def __len__(self):
    return len(self.ru_sents)

In [ ]:
tr_ru = T.Sequential(
    T.AddToken('<SOS>'),
    T.AddToken('<EOS>', begin=False),
    T.VocabTransform(ru_vocab),
    T.ToTensor(0),
    T.PadTransform(max_length=33, pad_value=0)
)
tr_en = T.Sequential(
    T.AddToken('<SOS>'),
    T.AddToken('<EOS>', begin=False),
    T.VocabTransform(en_vocab),
    T.ToTensor(0),
    T.PadTransform(max_length=33, pad_value=0)
)
train_dataset = RuEnDataset(X_train, y_train, [tr_ru, tr_en])
test_dataset = RuEnDataset(X_test, y_test, [tr_ru, tr_en])
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [ ]:
train_dataset[0]

(tensor([   2,   27,  677, 2206, 5594,    3,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0]),
 tensor([   2,    6,  148,  235,   22, 3980,    3,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0]))

<p class="task" id="4"></p>

4\. Опишите модель `Encoder`, которая возвращает скрытое состояние рекуррентного слоя в соотстветствии со следующей схемой. Пропустите через эту модель первые 16 предложений на русском языке и выведите размер полученного результата на экран. Результатом должен являться тензор размера `1 x batch_size x hidden_dim` (если используется один однонаправленный рекуррентный слой и `batch_first=True`).

* количество эмбеддингов равно количеству слов на русском языке;
* размерность эмбеддингов выберите самостоятельно;
* при создании слоя эмбеддингов укажите `padding_idx`;
* размер скрытого состояния рекуррентного слоя выберите самостоятельно.

![encoder](https://i0.wp.com/www.adeveloperdiary.com/wp-content/uploads/2020/10/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-1.png?w=815&ssl=1)

- [ ] Проверено на семинаре

In [ ]:
class Encoder(nn.Module):
  def __init__(self, emb_dim, hidden_size, n_ru):
    super().__init__()
    self.emb = nn.Embedding(num_embeddings=n_ru, embedding_dim=emb_dim, padding_idx=0)
    self.drop = nn.Dropout(p=0.2)
    self.rnn = nn.GRU(emb_dim, hidden_size, batch_first=True)

  def forward(self, X):
    e = self.emb(X)
    out = self.drop(e)
    out, h = self.rnn(out)
    return h # 1 x batch x hidden_size

<p class="task" id="5"></p>

5\. Опишите модель `Decoder`, которая возвращает прогноз (набор индексов слов на английском языке). Пропустите через эту модель тензор скрытых состояний кодировщика, полученный в предыдущей задачи, и выведите размер полученного результата на экран. Результатом должен являться тензор размера `batch_size x seq_len x n_en_words` (если используется один однонаправленный рекуррентный слой и `batch_first=True`).

* количество эмбеддингов равно количеству слов на английском языке;
* размер выходного слоя равен количеству слов на английском языке;
* размерность эмбеддингов выберите самостоятельно;
* при создании слоя эмбеддингов укажите `padding_idx`;
* размер скрытого состояния рекуррентного слоя выберите самостоятельно.

![decoder](https://i2.wp.com/www.adeveloperdiary.com/wp-content/uploads/2020/10/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-2.png?w=899&ssl=1)

- [ ] Проверено на семинаре

In [ ]:
class Decoder(nn.Module):
  def __init__(self, emb_dim, hidden_size, n_en):
    super().__init__()
    self.n_en_words = n_en
    self.emb = nn.Embedding(num_embeddings=n_en, embedding_dim=emb_dim, padding_idx=0)
    self.rnn = nn.GRUCell(emb_dim, hidden_size)
    self.fc = nn.Linear(in_features=hidden_size, out_features=n_en)

  def forward(self, encoder_out, labels):
    # labels: batch x seq_len - считаем, что в 0 столбце стоят sos
    # encoder: 1 x batch x hidden_size
    seq_len = labels.size(1)
    input_tokens = labels[:, 0]
    decoder_hidden = encoder_out[0]
    decoder_out = [th.zeros(size=(labels.size(0), 1, self.n_en_words)).to(device='cuda')]
    for i in range(1, seq_len):
      out = self.emb(input_tokens).relu() # batch x emb_dim
      decoder_hidden = self.rnn(out, decoder_hidden) # batch x dec_hidden
      out = self.fc(decoder_hidden) # batch x n_en
      decoder_out.append(out.unsqueeze(1))

      # teacher forcing
      input_tokens = labels[:, i]

    # вернуть прогнозы для каждого эл-та последовательности
    # batch x seq_len x n_en
    res = th.cat(decoder_out, dim=1)
    return res

<p class="task" id="6"></p>

6\. Объедините модели `Encoder` и `Decoder` в одну модель `EncoderDecoder`. Пропустите через эту модель первые 16 предложений на русском языке и выведите размер полученного результата на экран. Сделайте полученный результат двумерным, объединив две первые размерности: `batch_size * seq_len x n_en_words`. Выведите размерность полученного результата на экран.

- [ ] Проверено на семинаре

In [ ]:
class EncoderDecoder(nn.Module):
  def __init__(self, enc: Encoder, dec: Decoder):
    super().__init__()
    self.encoder = enc
    self.decoder = dec

  def forward(self, X, y):
    encoded = self.encoder(X)
    decoded = self.decoder(encoded, y)
    decoded = decoded.reshape(-1, self.decoder.n_en_words)
    return decoded

translation = EncoderDecoder(Encoder(100, 100, len(ru_vocab)), Decoder(100, 100, len(en_vocab)))

In [ ]:
x, y = train_dataset[:16]
x.shape, y.shape

(torch.Size([16, 33]), torch.Size([16, 33]))

In [ ]:
o = translation(x, y)
o.shape

torch.Size([528, 4234])

<p class="task" id="7"></p>

7\. Настройте модель, решив задачу классификации на основе прогнозов модели `EncoderDecoder`. Игнорируйте токен `<PAD>` при расчете ошибки. Во время обучения выводите на экран значения функции потерь для эпохи (на обучающем множестве), значение accuracy по токенам (на обучающем множестве) и пример перевода, сгенерированного моделью. После завершения обучения посчитайте BLEU для тестового множества.

- [ ] Проверено на семинаре

In [ ]:
n_epoch = 100
lr = 0.01
model = EncoderDecoder(Encoder(300, 150, len(ru_vocab)), Decoder(300, 150, len(en_vocab)))
model.to(device='cuda')
optimizer = optim.Adam(model.parameters(), lr=lr)
crit = nn.CrossEntropyLoss(ignore_index=0)

for epoch in range(1, n_epoch+1):
  acc = M.Accuracy(task="multiclass", num_classes=len(en_vocab), ignore_index=0)
  for X_b, y_b in train_loader:
    out = model(X_b.to(device='cuda'), y_b.to(device='cuda'))
    loss = crit(out, th.flatten(y_b.to(device='cuda')))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    acc.update(out.to(device='cpu').argmax(dim=1).detach(), th.flatten(y_b))

  if epoch % 10 == 0:
    print(f'{epoch=} loss={loss.item()} acc={acc.compute()}')
    x_ex, y_ex = next(iter(train_loader))
    x_ex, y_ex = x_ex[0], y_ex[0]
    x_ex, y_ex = x_ex[x_ex > 0].unsqueeze(0), y_ex[y_ex > 0].unsqueeze(0)
    out = model(x_ex.to(device='cuda'), y_ex.to(device='cuda'))
    out = out.detach().to(device='cpu')
    out = out.argmax(dim=1)
    print(f'true rus:\t{" ".join(ru_vocab.lookup_tokens(x_ex.tolist()[0])[1:-1])}')
    print(f'true eng:\t{" ".join(en_vocab.lookup_tokens(y_ex.tolist()[0])[1:-1])}')
    print(f'predicted:\t{" ".join(en_vocab.lookup_tokens(out.tolist())[1:-1])}')

epoch=10 loss=2.5806519985198975 acc=0.5399278998374939
true rus:	какая гора наивысшая точка кавказа
true eng:	which mountain is the highest point of the caucasus
predicted:	which city is the name point of the state
epoch=20 loss=1.7964651584625244 acc=0.7165220379829407
true rus:	кто изготовил первый печатный станок
true eng:	who made the first printing press
predicted:	who made the movie printing press
epoch=30 loss=1.2680152654647827 acc=0.7845678329467773
true rus:	когда открыта периодическая система менделеева
true eng:	when was the periodic table discovered
predicted:	when was the periodic table discovered
epoch=40 loss=1.0727688074111938 acc=0.8217413425445557
true rus:	какое воинское звание было у печорина
true eng:	what was pechorins military rank
predicted:	what was pechorins military rank
epoch=50 loss=1.0626951456069946 acc=0.8465501666069031
true rus:	в какому году начался период получивший название перестройка
true eng:	in what year did the period called perestroika begin

In [ ]:
import nltk

In [ ]:
BLEU_scores = []
for X_b, y_b in test_loader:
  out = model(X_b.to(device='cuda'), y_b.to(device='cuda'))
  out = out.detach().to(device='cpu')
  out = out.argmax(dim=1)

  y_b = th.flatten(y_b)
  out = th.flatten(out)
  not_pad_ind = (y_b > 0).nonzero(as_tuple=True)[0]
  y_b = y_b[not_pad_ind]
  out = out[not_pad_ind]
  hypothesis = y_b.tolist()
  reference = out.tolist()
  hypothesis = en_vocab.lookup_tokens(hypothesis)
  reference = en_vocab.lookup_tokens(reference)
  BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
  BLEU_scores.append(BLEUscore)
print(f'BLEU = {th.tensor(BLEU_scores).mean().item():.3f}')

BLEU = 0.183


## Обратная связь
- [x] Хочу получить обратную связь по решению